In [9]:
import torch
import schnetpack as spk
import math


import numpy as np
from numpy import savetxt

In [10]:
def print_molecule(props):
    
    # load x,y,z coordinates tensors
    x = props['_positions'][ :,0]
    y = props['_positions'][ :,1]
    z = props['_positions'][ :,2]
    x = x.numpy()
    y = y.numpy()
    z = z.numpy()
    
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 1:
            print('H',x[i],y[i],z[i])
        if props['_atomic_numbers'][i] == 6:
            print('C',x[i],y[i],z[i])
        if props['_atomic_numbers'][i] == 7:
            print('N',x[i],y[i],z[i])     
        if props['_atomic_numbers'][i] == 8:
            print('O',x[i],y[i],z[i])
        if props['_atomic_numbers'][i] == 9:
            print('F',x[i],y[i],z[i]) 
    print('')  
    return z

In [11]:
def hook_layer(self, inp_tensor, out_tensor):
    # Self is included and refers to the model class
    # Global allows us to utilize embedding_output outside the current function scope
    global layer_output
    #Update the embedding_output variable to be equal to our output tensor
    layer_output=out_tensor 

In [12]:
def convert_2D(number_of_atoms,layer_output):
    layer = np.zeros((number_of_atoms,30))
    for i in range(number_of_atoms):
        for j in range(30):
            layer[i][j] = layer_output[0][i][j]
    return layer

In [13]:
# Load QM9 dataset
from schnetpack.datasets import QM9

qm9data = QM9('./qm9.db', download=True, remove_uncharacterized=True)
# Load split file 
train, val, test = spk.data.train_test_split(qm9data,split_file='../../../data/trained_models/qm9_i6_30f_10000_5000/split.npz')

# Load atom ref data 
atomrefs = qm9data.get_atomref(QM9.U0)
# Define SchNet representation model

schnet = spk.representation.SchNet(
    n_atom_basis=30, n_filters=30, n_gaussians=20, n_interactions=6,
    cutoff=4. , cutoff_network=spk.nn.cutoff.CosineCutoff
)

# Define SchNet output model and property to be predicted

output_U0 = spk.atomistic.Atomwise(n_in=30, atomref=atomrefs[QM9.U0])

# Define atomistic model

model = spk.AtomisticModel(representation=schnet,output_modules=output_U0)

# Load saved checkpoint file
checkpoint_path = '../../../data/trained_models/qm9_i6_30f_10000_5000/qm9_benchmark.pth'
load_checkpoint = torch.load(checkpoint_path)

#qm9_i6_30f_20g-1000-500-4_300.pth
# load model's state dictionary from saved checkpoint
model.load_state_dict(load_checkpoint,strict=False)


number_of_inputs = 5000

#set up device for forward pass
device='cpu'

# load atoms converter 
converter = spk.data.AtomsConverter(device=device)

# load spk calculator
calculator = spk.interfaces.SpkCalculator(model=model, device=device, energy=QM9.U0)

data = np.zeros((1,30))
datahae = np.zeros((1))
dataoae = np.zeros((1))
datah = np.zeros((1,30))
for idx in range(number_of_inputs):
        
    # load data for molecule
    at, props = qm9data.get_properties(idx)
        
    # print molecule for identification
    print(idx)
    z = print_molecule(props)
    number_of_atoms=len(z)
        
    # set calculator on molecule
    at.set_calculator(calculator)
        
    # convert qm9 data to machine-readable form
    inputs = converter(at)
        
    #Instatiate layer output
    layer_output=None
        
    # Forward hook the model's interaction layer 
    model.representation.embedding.register_forward_hook(hook_layer)
        
    # Forward pass molecules through the model
    model(inputs)
    
    layer_output = layer_output.detach().numpy()
    
    from schnetpack.atomistic.output_modules import yi
    
    yi=yi.detach().numpy()

    #convert layer tensor to 2D array
    rows = convert_2D(number_of_atoms,layer_output)

    #save the vector of every oxygen atom encountered
    for i in range(number_of_atoms):
        if props['_atomic_numbers'][i] == 8:
            data = np.vstack((data,rows[i]))
            
    for i in range(number_of_atoms):
        if props['_atomic_numbers'][i] == 1:
            datah = np.vstack((datah,rows[i]))    
    for i in range(number_of_atoms):
        if props['_atomic_numbers'][i] == 1:
            datahae = np.vstack((datahae,yi[0][i]))
    for i in range(number_of_atoms):
        if props['_atomic_numbers'][i] == 8:
            dataoae = np.vstack((dataoae,yi[0][i]))            
    
print('DONE')            
savetxt('../../../data/dataO.csv',data,delimiter=',')
savetxt('../../../data/dataH.csv',datah,delimiter=',')
savetxt('../../../data/hae.csv',datahae,delimiter=',')
savetxt('../../../data/oae.csv',dataoae,delimiter=',')

None
0
C -2.8340169e-06 2.3049886e-06 -1.4378233e-07
H 0.014845718 -1.0918331 -0.0060250196
H 1.0244261 0.3779494 -0.007724565
H -0.52811974 0.36172476 -0.88464487
H -0.5111183 0.3521308 0.89839613



SpkCalculatorError: 'energy_U0' is not a property of your model. Please check the model properties!

In [6]:

from ase.io import read
atoms = read('../../../data/single-shot/etoh-eq.xyz')

converter = spk.data.AtomsConverter(device=device)
inputs = converter(atoms)

#Instatiate layer output
layer_output=None
        
# Forward hook the model's interaction layer 
model.representation.interactions[0].register_forward_hook(hook_layer)
        
# Forward pass molecules through the model
model(inputs)
    
layer_output = layer_output.detach().numpy()

print(layer_output)

None
[[[-1.45777333e+00  9.80087101e-01  2.25618935e+00 -1.22841740e+00
    5.18376923e+00 -7.78305352e-01 -1.23170924e+00  3.41663599e+00
   -9.27176923e-02 -4.03110790e+00 -5.43035626e-01 -2.79423141e+00
   -5.62853694e-01 -7.59041667e-01 -1.31645000e+00 -2.62874246e+00
    5.45372546e-01 -1.83699861e-01  1.26324427e+00  7.83805370e-01
    1.41192412e+00  7.60275960e-01 -1.56240201e+00 -1.43068194e+00
    1.32898057e+00 -9.62677479e-01 -1.78406978e+00 -3.31523752e+00
    1.53748441e+00 -5.27818441e-01]
  [-8.60149860e-01  6.64335072e-01  1.95967054e+00 -1.46611202e+00
    4.06217957e+00 -8.46186936e-01 -1.45423222e+00  3.23778176e+00
   -1.00398421e-01 -3.98679900e+00 -1.25431895e-01 -2.75682425e+00
   -2.16406673e-01 -9.15995359e-01 -1.19025147e+00 -3.14657831e+00
    6.67378128e-01  2.85841644e-01  1.40550995e+00  2.25806966e-01
    1.44665146e+00  3.34654927e-01 -1.42255306e+00 -1.25275171e+00
    1.57559800e+00 -9.78983283e-01 -1.77705014e+00 -2.71077156e+00
    1.19310820e+00 -5

/home/amerelsamman/Programs/anaconda3/lib/python3.7/site-packages/ase/atoms.py:972: VisibleDeprecationWarning: Use get_global_number_of_atoms() instead
  np.VisibleDeprecationWarning)


In [13]:
from ase.io import read

# load spk calculator
calculator = spk.interfaces.SpkCalculator(model=model, device=device, energy=QM9.U0)

atoms = read('../../../data/single-shot/etoh-dis1.xyz')

converter = spk.data.AtomsConverter(device=device)
inputs = converter(atoms)

model(inputs)


#Instatiate layer output
layer_output=None
        
# Forward hook the model's interaction layer 
model.representation.interactions[0].register_forward_hook(hook_layer)
        
# Forward pass molecules through the model
model(inputs)
    
layer_output = layer_output.detach().numpy()

print(layer_output)

[[[ 0.35496753 -0.5737914  -0.5637358  -0.38718292 -0.36334082
    0.3806352  -0.4056857   0.01782339  0.54130894 -0.35066447
    0.35857385 -0.37865785 -0.6438267  -0.39789596 -0.29758078
    0.03574721 -0.04382506  0.13773036 -0.3209071  -0.5804533
    0.48592576 -0.22727424 -0.15748699 -0.894549   -0.06014456
    0.33525354 -0.63482267 -0.20468843  0.02667927 -0.12938331]
  [-0.4202158  -0.31183016 -0.41321865  0.235461   -0.71231675
    0.22883967  0.35618293 -0.23944712  0.52235544 -0.10719116
    0.97808635 -0.45341164 -1.1624758  -0.10959862 -0.30854648
    0.10665571  0.44648337 -0.35972995  0.38170728 -0.1091295
    0.43652797 -0.80360705 -0.36896396 -0.23345491  0.99172777
   -0.15012194 -0.39222077 -0.05059789  0.09382409  0.2161303 ]
  [ 0.04614314 -0.13192683 -0.44135156 -0.18968    -0.527771
    0.1934692   0.12720388 -0.3418486   0.6817454  -0.2573214
    0.46396637 -0.28936338 -0.6801179  -0.7575228  -0.33928362
   -0.22108585  0.32412565 -0.22935149  0.03198098 -0.7560